In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_product_details(base_link, category_link, product_type_link):
    category_data = []
    
    for category in category_link:
        for product_type in product_type_link:
            category_url = f"{base_link}{category}{product_type}"
            category_page = requests.get(category_url)
            category_soup = BeautifulSoup(category_page.text, "html.parser")
            
            product_links = category_soup.find_all("a", onclick=True)  # Selettore da adattare se necessario
            
            for product_link in product_links:
                product_url = product_link['href']
                product_page = requests.get(product_url)
                product_soup = BeautifulSoup(product_page.text, "html.parser")
                
                product_details = {"category": category, "type": product_type}
                
                title_tag = product_soup.find("p", class_="product-title")
                product_details["name"] = title_tag.text.strip() if title_tag else "N/A"
                
                price_tag = product_soup.find("p", class_="price mt-2")
                product_details["price"] = price_tag.text.strip() if price_tag else "N/A"
                
                dettagli = product_soup.find_all("p", class_="mt-4")
                
                for dettaglio in dettagli:
                    if "Tacco:" in dettaglio.text and "Materiale esterno:" in dettaglio.text and 'Materiale interno:' in dettaglio.text and 'Materiale suola:' in dettaglio.text:
                        descrizione = dettaglio.text.strip()
                        product_details['description'] = descrizione
                
                category_data.append(product_details)
    
    return category_data

base_link = "https://www.kammi.it/"
category_link = ["scarpe-donna/"]
product_type_link = ["sneakers/", "sandali-bassi/", "stivali-stivaletti/", "zeppe/", "sandali-con-tacco/", "scarpe-con-tacco/", "scarpe-basse/"]

product_details = fetch_product_details(base_link, category_link, product_type_link)
product_df = pd.DataFrame(product_details)

product_df['category'] = product_df['category'].str.rstrip('/')
product_df['type'] = product_df['type'].str.rstrip('/')
product_df['price'] = product_df['price'].str.replace('€', '').str.replace(',', '.').astype(float)

def separa_descrizione(specifiche):
    dettagli = {}
    parti = specifiche.split('\n')
    for parte in parti:
        chiave_valore = parte.split(':')
        if len(chiave_valore) == 2:
            chiave, valore = chiave_valore
            chiave = chiave.strip()
            valore = valore.strip()
            dettagli[chiave] = valore
    return dettagli

descrizioni_dettagliate = product_df['description'].apply(separa_descrizione)
product_df['Tacco'] = descrizioni_dettagliate.apply(lambda x: x.get('Tacco', 'N/A'))
product_df['Materiale esterno'] = descrizioni_dettagliate.apply(lambda x: x.get('Materiale esterno', 'N/A'))
product_df['Materiale interno'] = descrizioni_dettagliate.apply(lambda x: x.get('Materiale interno', 'N/A'))
product_df['Materiale suola'] = descrizioni_dettagliate.apply(lambda x: x.get('Materiale suola', 'N/A'))
product_df = product_df.drop(columns=['description'])

product_df


,category,type,name,price,Tacco,Materiale esterno,Materiale interno,Materiale suola
0,scarpe-donna,sneakers,Sneakers 23180,89.9,0 cm,Pelle,Pelle,Sintetico
1,scarpe-donna,sneakers,Sneakers 9070,89.9,0 cm,Pelle,Pelle,Sintetico
2,scarpe-donna,sneakers,Sneakers 23039,95.0,0 cm,Pelle,Pelle,Gomma
3,scarpe-donna,sneakers,Sneakers 23039,79.9,4 cm,Pelle,Pelle,Gomma
4,scarpe-donna,sneakers,Sneakers 9070,89.9,0 cm,Pelle,Pelle,Gomma
...,...,...,...,...,...,...,...,...
86,scarpe-donna,scarpe-basse,Mocassino 8015,95.0,1 cm,Pelle,Pelle,Sintetico
87,scarpe-donna,scarpe-basse,Slingback 1052,59.9,0.5 cm,Tessuto,Sintetico,Sintetico
88,scarpe-donna,scarpe-basse,Spuntata 0712,59.9,0.5 cm,Tessuto,Sintetico,Sintetico
89,scarpe-donna,scarpe-basse,Mocassino Mina,79.9,2 cm,Sintetico,Pelle,Sintetico
